In [1]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
from scipy.stats import linregress as LR
import pandas as pd
from pandas.util.testing import assert_frame_equal
from sklearn.model_selection import train_test_split
pd.set_option("display.max_columns",200)

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from subprocess import call

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import recall_score
from imblearn.over_sampling import SMOTE

Using TensorFlow backend.
/home/crkellar@byu.local/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/crkellar@byu.local/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/crkellar@byu.local/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/crkellar@byu.local/.loc

In [10]:
#make a chronic patient table for this subset of conditions

# read in that pre-made table
df=pd.read_csv("../data_files/chronic_patients.csv")
df = df.dropna(axis = 0,how='any')
df.name = 'df'

In [11]:
X = df[['VMONTH', 'AGE', 'SEX', 'RACE', 'ETHNIC', 'RFV1', 'DIAG1', 'DIAG2',
              'DIAG3', 'DIAG4', 'DIAG5', 'MED1', 'PAYTYPE','YEAR','PATWT', 'chronic']]
y = df[['received']]

X_train, X_test, y_train, y_test = train_test_split(X, y)

In [12]:
y_train.received.value_counts()

0    341083
1     13464
Name: received, dtype: int64

In [15]:
#SMOTE (synthetic upsampling) the data
smt = SMOTE()
X_train.name='X_train'
y_train.name='y_train'
y_train.dtype=int
X_train, y_train = smt.fit_resample(X_train, y_train)

/home/crkellar@byu.local/.local/lib/python3.6/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [16]:
np.bincount(y_train)

array([341083, 341083])

In [18]:

model = RandomForestClassifier()

model.fit(X_train[['VMONTH', 'AGE', 'SEX', 'RACE', 'ETHNIC', 'RFV1', 'DIAG1', 'DIAG2',
              'DIAG3', 'DIAG4', 'DIAG5', 'MED1', 'PAYTYPE','YEAR']], np.array(y_train),sample_weight=X_train.PATWT.ravel())

y_predict = model.predict(X_test[['VMONTH', 'AGE', 'SEX', 'RACE', 'ETHNIC', 'RFV1', 'DIAG1', 'DIAG2',
              'DIAG3', 'DIAG4', 'DIAG5', 'MED1', 'PAYTYPE','YEAR']])#,sample_weight=X_wt2.PATWT.ravel())

#must include weights from the x_test, or this is invalid
accuracy_score(y_test[['received']], y_predict,sample_weight=X_test.PATWT.ravel())

0.9598791427244047

In [20]:
accuracy_score(y_test[['received']], y_predict)

0.961999610773123

In [21]:
pd.DataFrame(
    confusion_matrix(y_test[['received']], y_predict),
    columns=['Predicted Not Received', 'Predicted Received'],
    index=['True Not Received', 'True Received']
)

,Predicted Not Received,Predicted Received
True Not Received,112508,1115
True Received,3376,1184


In [24]:
accuracy_score(y_test[['received']], y_predict)

0.961999610773123

In [23]:
recall_score(y_test[['received']], y_predict)

0.2596491228070175